In [1]:
import numpy as np
import pyreadr
import math
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import textwrap

from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

from utils import pca

In [2]:
path = "./gss_16.rda"

In [3]:
data = pyreadr.read_r(path)

In [4]:
df = data["gss16"]

In [5]:
columns_to_check = ['educ', 'wrkstat']
df.dropna(subset=columns_to_check, inplace=True)

In [6]:
value_to_index = {}

for col in df.columns.tolist():
    try:
        unique_set = np.unique(df[col].to_numpy()).tolist()
        unique_set = [x for x in unique_set if not math.isnan(x)]
        print(col, unique_set)
        unique_set.sort()
        value_to_index[col] = dict(zip(unique_set, unique_set))
    except:
        #print(col, list(set(df[col].tolist())))
        unique_set = list(set(df[col].tolist()))
        chosens = []
        for i in unique_set:
            if isinstance(i, float):
                if math.isnan(i):
                    continue
            #print(type(i), i)
            chosens.append(i)
        unique_set = chosens
        
        print(col, unique_set)
        unique_set.sort()
    
        if col == "advfront":
            value_to_index[col] = {'Strongly disagree': 1, 'Disagree': 2, 'Dont know': 3, 'Agree': 4, 'Strongly agree': 5}
        elif col == "polviews":
            value_to_index[col] = {'Extrmly conservative': 1, 'Conservative': 2, 'Slghtly conservative': 3, 'Moderate': 4, 'Slightly liberal': 5, 'Liberal': 6, 'Extremely liberal': 7}
        elif col == "educ" or col == "wrkstat":
            value_to_index[col] = dict(zip(unique_set, range(len(unique_set))))
        else:
            value_to_index[col] = dict(zip(unique_set, range(1, len(unique_set) + 1)))
        
print(value_to_index)

harass5 ['Does not apply (i do not have a job/superior/co-worker)', 'Yes', 'No']
emailmin [0.0, 1.0, 2.0, 5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 40.0, 45.0, 48.0, 50.0, 59.0]
emailhr [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 12.0, 14.0, 15.0, 16.0, 17.0, 18.0, 20.0, 21.0, 22.0, 24.0, 25.0, 28.0, 30.0, 32.0, 35.0, 36.0, 40.0, 42.0, 44.0, 45.0, 47.0, 48.0, 50.0, 56.0, 60.0, 70.0, 72.0, 75.0, 90.0, 100.0]
educ [0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0]
polviews ['Slghtly conservative', 'Slightly liberal', 'Extremely liberal', 'Liberal', 'Conservative', 'Extrmly conservative', 'Moderate']
advfront ['Strongly agree', 'Strongly disagree', 'Dont know', 'Disagree', 'Agree']
snapchat ['Yes', 'No']
instagrm ['Yes', 'No']
wrkstat ['Retired', 'Unempl, laid off', 'Working fulltime', 'Temp not working', 'School', 'Keeping house', 'Working parttime', 'Other']
{'harass5': {'Does not apply (i do not have a job/superior

In [7]:
for col in df.columns.tolist():
    df[col] = df[col].map(value_to_index[col])

In [8]:
df

,harass5,emailmin,emailhr,educ,polviews,advfront,snapchat,instagrm,wrkstat
0,NaN,0.0,12.0,16.0,4.0,5.0,NaN,NaN,6
1,NaN,30.0,0.0,12.0,6.0,2.0,1.0,1.0,6
2,2.0,NaN,NaN,16.0,2.0,NaN,1.0,1.0,2
3,NaN,10.0,0.0,12.0,4.0,2.0,NaN,NaN,7
4,2.0,NaN,NaN,18.0,5.0,NaN,2.0,2.0,7
...,...,...,...,...,...,...,...,...,...
2862,NaN,0.0,20.0,20.0,7.0,4.0,1.0,2.0,6
2863,NaN,0.0,2.0,15.0,2.0,2.0,NaN,NaN,6
2864,2.0,0.0,0.0,14.0,4.0,NaN,NaN,NaN,0
2865,NaN,0.0,1.0,14.0,3.0,4.0,NaN,NaN,6


In [9]:
categorical_columns = ['harass5', 'polviews', 'advfront', 'snapchat', 'instagrm', 'wrkstat']

df[categorical_columns] = df[categorical_columns].fillna(0)
numerical_columns = ['emailmin', 'emailhr', 'educ', 'polviews']

df[numerical_columns] = df[numerical_columns].fillna(-1)

In [10]:
df

,harass5,emailmin,emailhr,educ,polviews,advfront,snapchat,instagrm,wrkstat
0,0.0,0.0,12.0,16.0,4.0,5.0,0.0,0.0,6
1,0.0,30.0,0.0,12.0,6.0,2.0,1.0,1.0,6
2,2.0,-1.0,-1.0,16.0,2.0,0.0,1.0,1.0,2
3,0.0,10.0,0.0,12.0,4.0,2.0,0.0,0.0,7
4,2.0,-1.0,-1.0,18.0,5.0,0.0,2.0,2.0,7
...,...,...,...,...,...,...,...,...,...
2862,0.0,0.0,20.0,20.0,7.0,4.0,1.0,2.0,6
2863,0.0,0.0,2.0,15.0,2.0,2.0,0.0,0.0,6
2864,2.0,0.0,0.0,14.0,4.0,0.0,0.0,0.0,0
2865,0.0,0.0,1.0,14.0,3.0,4.0,0.0,0.0,6


In [11]:
df["wrkstat"] = df["wrkstat"].astype(np.int16)

In [12]:
df

,harass5,emailmin,emailhr,educ,polviews,advfront,snapchat,instagrm,wrkstat
0,0.0,0.0,12.0,16.0,4.0,5.0,0.0,0.0,6
1,0.0,30.0,0.0,12.0,6.0,2.0,1.0,1.0,6
2,2.0,-1.0,-1.0,16.0,2.0,0.0,1.0,1.0,2
3,0.0,10.0,0.0,12.0,4.0,2.0,0.0,0.0,7
4,2.0,-1.0,-1.0,18.0,5.0,0.0,2.0,2.0,7
...,...,...,...,...,...,...,...,...,...
2862,0.0,0.0,20.0,20.0,7.0,4.0,1.0,2.0,6
2863,0.0,0.0,2.0,15.0,2.0,2.0,0.0,0.0,6
2864,2.0,0.0,0.0,14.0,4.0,0.0,0.0,0.0,0
2865,0.0,0.0,1.0,14.0,3.0,4.0,0.0,0.0,6


In [13]:
categorical_columns = ['harass5', 'snapchat', 'instagrm', 'advfront']
numerical_columns = ['emailmin', 'emailhr', 'educ']

df_encoded = pd.get_dummies(df, columns=categorical_columns, drop_first=False)
scaler = MinMaxScaler()
#scaler = StandardScaler()

df_encoded[numerical_columns] = scaler.fit_transform(df_encoded[numerical_columns])
#df_encoded = scaler.fit_transform(df_encoded)

df_encoded

,emailmin,emailhr,educ,polviews,wrkstat,harass5_0.0,harass5_1.0,harass5_2.0,harass5_3.0,snapchat_0.0,...,snapchat_2.0,instagrm_0.0,instagrm_1.0,instagrm_2.0,advfront_0.0,advfront_1.0,advfront_2.0,advfront_3.0,advfront_4.0,advfront_5.0
0,0.016667,0.128713,0.80,4.0,6,1,0,0,0,1,...,0,1,0,0,0,0,0,0,0,1
1,0.516667,0.009901,0.60,6.0,6,1,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
2,0.000000,0.000000,0.80,2.0,2,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
3,0.183333,0.009901,0.60,4.0,7,1,0,0,0,1,...,0,1,0,0,0,0,1,0,0,0
4,0.000000,0.000000,0.90,5.0,7,0,0,1,0,0,...,1,0,0,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2862,0.016667,0.207921,1.00,7.0,6,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
2863,0.016667,0.029703,0.75,2.0,6,1,0,0,0,1,...,0,1,0,0,0,0,1,0,0,0
2864,0.016667,0.009901,0.70,4.0,0,0,0,1,0,1,...,0,1,0,0,1,0,0,0,0,0
2865,0.016667,0.019802,0.70,3.0,6,1,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0


In [14]:
y = df_encoded["wrkstat"]
y = y.to_numpy()

x = df_encoded.drop("wrkstat", axis=1).to_numpy()
_, reduced_x = pca(x=x, alpha=0.95)

x.shape, y, np.real(reduced_x)

((2858, 20),
 array([6, 6, 2, ..., 0, 6, 2], dtype=int16),
 array([[ 3.95758801, -1.47443099,  0.25092183, ...,  0.61484668,
          0.37876778, -0.03802543],
        [ 6.01839966, -0.08468529, -1.02816384, ...,  0.07139986,
          0.22944504, -0.10936024],
        [ 2.0681682 ,  1.26113052,  0.07934565, ..., -0.2598679 ,
          0.43035821, -0.05632102],
        ...,
        [ 3.96125115, -0.27916221,  1.49480334, ..., -0.3555741 ,
          0.36042437, -0.11201655],
        [ 2.91990449, -1.54120793,  0.04820226, ..., -0.73364502,
          0.19581022, -0.07565114],
        [ 3.06604559,  1.21043411,  0.10117701, ..., -0.2840839 ,
          0.42838743, -0.0539307 ]]))

In [15]:
X_train, X_test, y_train, y_test = train_test_split(np.real(reduced_x), y, test_size=0.4, random_state=42)

In [16]:
model = LogisticRegression(multi_class='multinomial', solver='saga')
model.fit(X_train, y_train)

LogisticRegression(multi_class='multinomial', solver='saga')

In [17]:
y_pred = model.predict(X_test)
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       111
           1       0.00      0.00      0.00        33
           2       0.44      0.07      0.12       238
           3       0.00      0.00      0.00        33
           4       0.00      0.00      0.00        21
           5       0.00      0.00      0.00        40
           6       0.48      0.98      0.65       542
           7       0.00      0.00      0.00       126

    accuracy                           0.48      1144
   macro avg       0.12      0.13      0.10      1144
weighted avg       0.32      0.48      0.33      1144



c:\Users\Thanh_Tuyet\.conda\envs\pytorch_tensorflow\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Thanh_Tuyet\.conda\envs\pytorch_tensorflow\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Thanh_Tuyet\.conda\envs\pytorch_tensorflow\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier